# 张量数据结构

张量和计算图是 TensorFlow 的核心概念。Tensorflow 的基本数据结构是张量 Tensor。张量即多维数组。Tensorflow 的张量和 numpy 中的 array 很类似。从行为特性来看，有两种类型的张量，常量constant 和变量 Variable。常量的值在计算图中不可以被重新赋值，变量可以在计算图中用 assign 等算子重新赋值。

In [1]:
import numpy as np
import tensorflow as tf

## 常量张量

张量的数据类型和 numpy.array 基本一一对应。



In [ ]:

i = tf.constant(1) # tf.int32 类型常量
l = tf.constant(1,dtype = tf.int64) # tf.int64 类型常量
f = tf.constant(1.23) #tf.float32 类型常量
d = tf.constant(3.14,dtype = tf.double) # tf.double 类型常量
s = tf.constant("hello world") # tf.string类型常量
b = tf.constant(True) #tf.bool类型常量


print(tf.int64 == np.int64) 
print(tf.bool == np.bool)
print(tf.double == np.float64)
print(tf.string == np.unicode) # tf.string类型和np.unicode类型不等价

True
True
True
False


In [ ]:
不同类型的数据可以用不同维度(rank)的张量来表示。标量为0维张量，向量为1维张量，矩阵为2维张量。
彩色图像有rgb三个通道，可以表示为3维张量。视频还有时间维，可以表示为4维张量。


In [ ]:
scalar = tf.constant(True)  # 标量，0 维张量

print(tf.rank(scalar))
print(scalar.numpy().ndim)  # tf.rank的作用和numpy的ndim方法相同

vector = tf.constant([1.0,2.0,3.0,4.0]) # 向量，1维张量

print(tf.rank(vector))
print(np.ndim(vector.numpy()))

matrix = tf.constant([[1.0,2.0],[3.0,4.0]]) # 矩阵, 2维张量

print(tf.rank(matrix).numpy())
print(np.ndim(matrix))

tensor3 = tf.constant([[[1.0,2.0],[3.0,4.0]],[[5.0,6.0],[7.0,8.0]]])  # 3维张量
print(tensor3)
print(tf.rank(tensor3))

tensor4 = tf.constant([[[[1.0,1.0],[2.0,2.0]],[[3.0,3.0],[4.0,4.0]]],
                        [[[5.0,5.0],[6.0,6.0]],[[7.0,7.0],[8.0,8.0]]]])  # 4维张量
print(tensor4)
print(tf.rank(tensor4))


可以用tf.cast改变张量的数据类型。可以用numpy方法将tensorflow中的张量转化成numpy中的张量。可以用shape方法查看张量的尺寸。

In [ ]:
h = tf.constant([123,456],dtype = tf.int32)
f = tf.cast(h,tf.float32)
print(h.dtype, f.dtype)

y = tf.constant([[1.0,2.0],[3.0,4.0]])
print(y.numpy()) #转换成np.array
print(y.shape)

u = tf.constant(u"你好 世界")
print(u.numpy())  
print(u.numpy().decode("utf-8"))

## 变量张量

模型中需要被训练的参数一般被设置成变量。



In [ ]:
# 常量值不可以改变，常量的重新赋值相当于创造新的内存空间
c = tf.constant([1.0,2.0])
print(c)
print(id(c))
c = c + tf.constant([1.0,1.0])
print(c)
print(id(c))

# 变量的值可以改变，可以通过assign, assign_add等方法给变量重新赋值
v = tf.Variable([1.0,2.0],name = "v")
print(v)
print(id(v))
v.assign_add([1.0,1.0])
print(v)
print(id(v))

# 张量的结构操作

张量的操作主要包括张量的结构操作和张量的数学运算。张量结构操作诸如：张量创建，索引切片，维度变换，合并分割。张量数学运算主要有：标量运算，向量运算，矩阵运算。另外我们会介绍张量运算的广播机制。

## 张量创建

In [3]:
a = tf.constant([1,2,3],dtype = tf.float32)
tf.print(a)

b = tf.range(1,10,delta = 2)
tf.print(b)

c = tf.linspace(0.0,2*3.14,100)
tf.print(c)

d = tf.zeros([3,3])
tf.print(d)

a = tf.ones([3,3])
b = tf.zeros_like(a,dtype= tf.float32)
tf.print(a)
tf.print(b)

b = tf.fill([3,2],5)
tf.print(b)

# 均匀分布随机
tf.random.set_seed(1.0)
a = tf.random.uniform([5],minval=0,maxval=10)
tf.print(a)

# 正态分布随机
b = tf.random.normal([3,3],mean=0.0,stddev=1.0)
tf.print(b)

# 正态分布随机，剔除2倍方差以外数据重新生成
c = tf.random.truncated_normal((5,5), mean=0.0, stddev=1.0, dtype=tf.float32)
tf.print(c)

# 特殊矩阵
I = tf.eye(3,3) #单位矩阵
tf.print(I)
tf.print(" ")
t = tf.linalg.diag([1,2,3]) #对角阵
tf.print(t)

[1 2 3]
[1 3 5 7 9]
[0 0.0634343475 0.126868695 ... 6.15313148 6.21656609 6.28]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
[[1 1 1]
 [1 1 1]
 [1 1 1]]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
[[5 5]
 [5 5]
 [5 5]]
[1.65130854 9.01481247 6.30974197 4.34546089 2.9193902]


## 索引切片

张量的索引切片方式和 numpy 几乎是一样的。切片时支持缺省参数和省略号。对于 tf.Variable,可以通过索引和切片对部分元素进行修改。对于提取张量的连续子区域，也可以使用 tf.slice，此外，对于不规则的切片提取,可以使用tf.gather,tf.gather_nd,tf.boolean_mask。

tf.boolean_mask功能最为强大，它可以实现tf.gather,tf.gather_nd的功能，并且 tf.boolean_mask还可以实现布尔索引。如果要通过修改张量的某些元素得到新的张量，可以使用 tf.where，tf.scatter_nd。

In [4]:
tf.random.set_seed(3)
t = tf.random.uniform([5,5],minval=0,maxval=10,dtype=tf.int32)
tf.print(t)

[[4 7 4 2 9]
 [9 1 2 4 7]
 [7 2 7 4 0]
 [9 6 9 7 2]
 [3 7 0 0 3]]


In [ ]:
# 第0行
tf.print(t[0])

# 倒数第一行
tf.print(t[-1])

# 第1行第3列
tf.print(t[1,3])
tf.print(t[1][3])

# 第1行至第3行
tf.print(t[1:4,:])
tf.print(tf.slice(t,[1,0],[3,5])) #tf.slice(input,begin_vector,size_vector)

# 第1行至最后一行，第0列到最后一列每隔两列取一列
tf.print(t[1:4,:4:2])

In [ ]:
# 对变量来说，还可以使用索引和切片修改部分元素
x = tf.Variable([[1,2],[3,4]],dtype = tf.float32)
x[1,:].assign(tf.constant([0.0,0.0]))
tf.print(x)

a = tf.random.uniform([3,3,3],minval=0,maxval=10,dtype=tf.int32)
tf.print(a)

# 省略号可以表示多个冒号
tf.print(a[...,1])

以上切片方式相对规则，对于不规则的切片提取,可以使用 tf.gather,tf.gather_nd,tf.boolean_mask。考虑班级成绩册的例子，有4个班级，每个班级10个学生，每个学生7门科目成绩。可以用一个4×10×7的张量来表示。

In [5]:
scores = tf.random.uniform((4,10,7),minval=0,maxval=100,dtype=tf.int32)
tf.print(scores)

[[[27 83 99 ... 70 67 79]
  [6 97 31 ... 3 40 48]
  [71 63 31 ... 44 30 86]
  ...
  [10 94 95 ... 89 1 13]
  [61 6 81 ... 24 46 10]
  [61 20 89 ... 33 95 78]]

 [[26 1 5 ... 2 62 29]
  [40 0 68 ... 37 15 28]
  [70 60 82 ... 72 64 1]
  ...
  [49 60 74 ... 18 87 35]
  [2 38 41 ... 66 8 74]
  [37 12 60 ... 57 29 51]]

 [[33 79 23 ... 5 24 45]
  [66 50 83 ... 32 30 58]
  [47 91 21 ... 59 81 1]
  ...
  [98 93 66 ... 55 67 69]
  [71 93 26 ... 77 11 63]
  [90 75 74 ... 7 68 14]]

 [[72 1 53 ... 97 57 28]
  [24 5 45 ... 41 86 23]
  [99 68 54 ... 97 65 85]
  ...
  [94 4 69 ... 88 17 93]
  [28 48 9 ... 73 32 39]
  [18 2 40 ... 51 37 24]]]


In [6]:
# 抽取每个班级第0个学生，第5个学生，第9个学生的全部成绩
p = tf.gather(scores,[0,5,9],axis=1)
tf.print(p)

# 抽取每个班级第0个学生，第5个学生，第9个学生的第1门课程，第3门课程，第6门课程成绩
q = tf.gather(tf.gather(scores,[0,5,9],axis=1),[1,3,6],axis=2)
tf.print(q)


# 抽取第0个班级第0个学生，第2个班级的第4个学生，第3个班级的第6个学生的全部成绩
#indices的长度为采样样本的个数，每个元素为采样位置的坐标
s = tf.gather_nd(scores,indices = [(0,0),(2,4),(3,6)])
s

[[[27 83 99 ... 70 67 79]
  [96 78 75 ... 2 3 49]
  [61 20 89 ... 33 95 78]]

 [[26 1 5 ... 2 62 29]
  [16 51 72 ... 11 80 78]
  [37 12 60 ... 57 29 51]]

 [[33 79 23 ... 5 24 45]
  [46 73 64 ... 27 39 13]
  [90 75 74 ... 7 68 14]]

 [[72 1 53 ... 97 57 28]
  [84 99 85 ... 79 57 62]
  [18 2 40 ... 51 37 24]]]
[[[83 9 79]
  [78 17 49]
  [20 9 78]]

 [[1 17 29]
  [51 94 78]
  [12 45 51]]

 [[79 82 45]
  [73 71 13]
  [75 12 14]]

 [[1 87 28]
  [99 71 62]
  [2 74 24]]]


以上tf.gather和tf.gather_nd的功能也可以用tf.boolean_mask来实现。



In [7]:
# 抽取每个班级第0个学生，第5个学生，第9个学生的全部成绩
p = tf.boolean_mask(scores,[True,False,False,False,False,
                            True,False,False,False,True],axis=1)
tf.print(p)

# 抽取第0个班级第0个学生，第2个班级的第4个学生，第3个班级的第6个学生的全部成绩
s = tf.boolean_mask(scores,
    [[True,False,False,False,False,False,False,False,False,False],
     [False,False,False,False,False,False,False,False,False,False],
     [False,False,False,False,True,False,False,False,False,False],
     [False,False,False,False,False,False,True,False,False,False]])
tf.print(s)

# 利用tf.boolean_mask可以实现布尔索引

# 找到矩阵中小于0的元素
c = tf.constant([[-1,1,-1],[2,2,-2],[3,-3,3]],dtype=tf.float32)
tf.print(c,"\n")

tf.print(tf.boolean_mask(c,c<0),"\n") 
tf.print(c[c<0]) #布尔索引，为boolean_mask的语法糖形式

[[[27 83 99 ... 70 67 79]
  [96 78 75 ... 2 3 49]
  [61 20 89 ... 33 95 78]]

 [[26 1 5 ... 2 62 29]
  [16 51 72 ... 11 80 78]
  [37 12 60 ... 57 29 51]]

 [[33 79 23 ... 5 24 45]
  [46 73 64 ... 27 39 13]
  [90 75 74 ... 7 68 14]]

 [[72 1 53 ... 97 57 28]
  [84 99 85 ... 79 57 62]
  [18 2 40 ... 51 37 24]]]
[[27 83 99 ... 70 67 79]
 [44 94 85 ... 65 80 71]
 [88 35 53 ... 51 2 89]]
[[-1 1 -1]
 [2 2 -2]
 [3 -3 3]] 

[-1 -1 -2 -3] 

[-1 -1 -2 -3]


以上这些方法仅能提取张量的部分元素值，但不能更改张量的部分元素值得到新的张量。如果要通过修改张量的部分元素值得到新的张量，可以使用tf.where和tf.scatter_nd。

- tf.where可以理解为if的张量版本，此外它还可以用于找到满足条件的所有元素的位置坐标。

- tf.scatter_nd的作用和tf.gather_nd有些相反，tf.gather_nd用于收集张量的给定位置的元素，而tf.scatter_nd可以将某些值插入到一个给定shape的全0的张量的指定位置处。

In [8]:
# 找到张量中小于0的元素,将其换成np.nan得到新的张量
# tf.where和np.where作用类似，可以理解为if的张量版本

c = tf.constant([[-1,1,-1],[2,2,-2],[3,-3,3]],dtype=tf.float32)
d = tf.where(c<0,tf.fill(c.shape,np.nan),c) 
d

# 如果where只有一个参数，将返回所有满足条件的位置坐标
indices = tf.where(c<0)
indices

# 将张量的第[0,0]和[2,1]两个位置元素替换为0得到新的张量
d = c - tf.scatter_nd([[0,0],[2,1]],[c[0,0],c[2,1]],c.shape)
d

# scatter_nd的作用和gather_nd有些相反
#可以将某些值插入到一个给定shape的全0的张量的指定位置处。
indices = tf.where(c<0)
tf.scatter_nd(indices,tf.gather_nd(c,indices),c.shape)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-1.,  0., -1.],
       [ 0.,  0., -2.],
       [ 0., -3.,  0.]], dtype=float32)>

## 维度变换

维度变换相关函数主要有 tf.reshape, tf.squeeze, tf.expand_dims, tf.transpose.

- tf.reshape 可以改变张量的形状。
- tf.squeeze 可以减少维度。
- tf.expand_dims 可以增加维度。
- tf.transpose 可以交换维度。

tf.reshape可以改变张量的形状，但是其本质上不会改变张量元素的存储顺序，所以，该操作实际上非常迅速，并且是可逆的。

In [9]:
a = tf.random.uniform(shape=[1,3,3,2],
                      minval=0,maxval=255,dtype=tf.int32)
tf.print(a.shape)
tf.print(a)

TensorShape([1, 3, 3, 2])
[[[[242 247]
   [26 215]
   [237 156]]

  [[229 38]
   [26 154]
   [26 128]]

  [[148 66]
   [147 128]
   [146 173]]]]


In [10]:
# 改成 （3,6）形状的张量
b = tf.reshape(a,[3,6])
tf.print(b.shape)
tf.print(b)

TensorShape([3, 6])
[[242 247 26 215 237 156]
 [229 38 26 154 26 128]
 [148 66 147 128 146 173]]


In [ ]:
# 改回成 [1,3,3,2] 形状的张量
c = tf.reshape(b,[1,3,3,2])
tf.print(c)

如果张量在某个维度上只有一个元素，利用tf.squeeze可以消除这个维度。和tf.reshape相似，它本质上不会改变张量元素的存储顺序。张量的各个元素在内存中是线性存储的，其一般规律是，同一层级中的相邻元素的物理地址也相邻。



In [11]:
s = tf.squeeze(a)
tf.print(s.shape)
tf.print(s)

d = tf.expand_dims(s,axis=0) #在第0维插入长度为1的一个维度
d

TensorShape([3, 3, 2])
[[[242 247]
  [26 215]
  [237 156]]

 [[229 38]
  [26 154]
  [26 128]]

 [[148 66]
  [147 128]
  [146 173]]]


tf.transpose可以交换张量的维度，与tf.reshape不同，它会改变张量元素的存储顺序。tf.transpose常用于图片存储格式的变换上。



In [12]:
# Batch,Height,Width,Channel
a = tf.random.uniform(shape=[100,600,600,4],minval=0,maxval=255,dtype=tf.int32)
tf.print(a.shape)

# 转换成 Channel,Height,Width,Batch
s= tf.transpose(a,perm=[3,1,2,0])
tf.print(s.shape)

TensorShape([100, 600, 600, 4])
TensorShape([4, 600, 600, 100])


## 合并分割

和numpy类似，可以用tf.concat和tf.stack方法对多个张量进行合并，可以用tf.split方法把一个张量分割成多个张量。tf.concat和tf.stack有略微的区别，tf.concat是连接，不会增加维度，而tf.stack是堆叠，会增加维度。



In [13]:
a = tf.constant([[1.0,2.0],[3.0,4.0]])
b = tf.constant([[5.0,6.0],[7.0,8.0]])
c = tf.constant([[9.0,10.0],[11.0,12.0]])

tf.concat([a,b,c],axis = 0)

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[ 1.,  2.],
       [ 3.,  4.],
       [ 5.,  6.],
       [ 7.,  8.],
       [ 9., 10.],
       [11., 12.]], dtype=float32)>

In [16]:
tf.concat([a,b,c],axis = 1)


<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[ 1.,  2.,  5.,  6.,  9., 10.],
       [ 3.,  4.,  7.,  8., 11., 12.]], dtype=float32)>

In [17]:
tf.stack([a,b,c])


<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
array([[[ 1.,  2.],
        [ 3.,  4.]],

       [[ 5.,  6.],
        [ 7.,  8.]],

       [[ 9., 10.],
        [11., 12.]]], dtype=float32)>

In [18]:
tf.stack([a,b,c],axis=1)


<tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
array([[[ 1.,  2.],
        [ 5.,  6.],
        [ 9., 10.]],

       [[ 3.,  4.],
        [ 7.,  8.],
        [11., 12.]]], dtype=float32)>

In [20]:
a = tf.constant([[1.0,2.0],[3.0,4.0]])
b = tf.constant([[5.0,6.0],[7.0,8.0]])
c = tf.constant([[9.0,10.0],[11.0,12.0]])

c = tf.concat([a,b,c],axis = 0)

tf.split是tf.concat的逆运算，可以指定分割份数平均分割，也可以通过指定每份的记录数量进行分割。



In [21]:
#tf.split(value,num_or_size_splits,axis)
tf.split(c,3,axis = 0)  #指定分割份数，平均分割

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 2.],
        [3., 4.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[5., 6.],
        [7., 8.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[ 9., 10.],
        [11., 12.]], dtype=float32)>]

In [ ]:
tf.split(c,[2,2,2],axis = 0) #指定每份的记录数量
